<center>

# $\textbf{Merge Files}$

<center>

### $\textbf{Code}$

In [9]:
from pyspark.sql import SparkSession
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf "spark.executor.extraJavaOptions=-XX:ReservedCodeCacheSize=2048m" --conf "spark.driver.extraJavaOptions=-XX:ReservedCodeCacheSize=2048m" pyspark-shell'

In [10]:
spark = SparkSession.builder.appName('MergeFiles').master("local").enableHiveSupport().getOrCreate()
spark

In [11]:
# Set paths to the Parquet files
file_paths = ["FilesParquet/Covid.parquet", "FilesParquet/GDP.parquet", "FilesParquet/Inflation.parquet",
              "FilesParquet/Migration.parquet", "FilesParquet/Population.parquet", "FilesParquet/Tax.parquet", "FilesParquet/Unemployment.parquet"]

# Read Parquet files into DataFrames
dfs = [spark.read.parquet(file_path) for file_path in file_paths]

# Merge or outer join the DataFrames
merged_df = dfs[0]
#for i, df in enumerate(dfs):
    # Generate new column names with a suffix indicating the iteration number
#    renamed_columns = [(c, f"{c}_df{i+1}") for c in df.columns if c == "year"]
    # Rename columns in the DataFrame
#    for old_name, new_name in renamed_columns:
#        df = df.withColumnRenamed(old_name, new_name)
    # Update the DataFrame in the list
#    dfs[i] = df

for df in dfs[1:]:
    merged_df = merged_df.join(df, on=["country","year"], how="outer")

merged_df.show()

+------------------+----+-----------+---------+---------+---------+----------+----+------------+
|           country|year|covid_cases|      gdp|inflation|migration|population| tax|unemployment|
+------------------+----+-----------+---------+---------+---------+----------+----+------------+
|           ASEAN-5|2011|       null| 4742.031|      4.7|     null|      null|null|        null|
|           ASEAN-5|2012|       null| 4934.317|      3.4|     null|      null|null|        null|
|           ASEAN-5|2013|       null| 5009.992|      4.2|     null|      null|null|        null|
|           ASEAN-5|2014|       null| 4941.123|      4.3|     null|      null|null|        null|
|           ASEAN-5|2015|       null| 4715.428|      3.1|     null|      null|null|        null|
|           ASEAN-5|2016|       null| 4889.034|      2.0|     null|      null|null|        null|
|           ASEAN-5|2017|       null| 5213.511|      2.8|     null|      null|null|        null|
|           ASEAN-5|2018|     

In [12]:
#Storing this dataframe in parquet
merged_df.write.mode("overwrite").parquet("FinalOuterParquet")
spark.read.parquet("FinalOuterParquet").show()
spark.stop()

+------------------+----+-----------+---------+---------+---------+----------+----+------------+
|           country|year|covid_cases|      gdp|inflation|migration|population| tax|unemployment|
+------------------+----+-----------+---------+---------+---------+----------+----+------------+
|           ASEAN-5|2011|       null| 4742.031|      4.7|     null|      null|null|        null|
|           ASEAN-5|2012|       null| 4934.317|      3.4|     null|      null|null|        null|
|           ASEAN-5|2013|       null| 5009.992|      4.2|     null|      null|null|        null|
|           ASEAN-5|2014|       null| 4941.123|      4.3|     null|      null|null|        null|
|           ASEAN-5|2015|       null| 4715.428|      3.1|     null|      null|null|        null|
|           ASEAN-5|2016|       null| 4889.034|      2.0|     null|      null|null|        null|
|           ASEAN-5|2017|       null| 5213.511|      2.8|     null|      null|null|        null|
|           ASEAN-5|2018|     